# Imports

In [1]:
!pip install datasets

In [2]:
!pip install transformers

In [3]:
!pip install evaluate

In [4]:
from sklearn.model_selection import train_test_split
from datasets import Dataset, DatasetDict, load_dataset, interleave_datasets, load_from_disk
from transformers import AutoModelForSeq2SeqLM, AutoTokenizer, GenerationConfig, TrainingArguments, Trainer
import torch
import time
import evaluate
import pandas as pd
import numpy as np

import warnings
warnings.filterwarnings("ignore")

# Model Load

In [5]:
!unzip "/content/drive/MyDrive/Dirllm/distilled_model.zip" -d "/content/"

Archive:  /content/drive/MyDrive/Dirllm/distilled_model.zip
   creating: /content/distilled/
  inflating: /content/distilled/config.json  
  inflating: /content/distilled/generation_config.json  
  inflating: /content/distilled/model.safetensors  


In [6]:
!unzip "/content/drive/MyDrive/Dirllm/Teacher_dataset.zip" -d "/content/"

Archive:  /content/drive/MyDrive/Dirllm/Teacher_dataset.zip
   creating: /content/Teacher_dataset/
 extracting: /content/Teacher_dataset/dataset_dict.json  
   creating: /content/Teacher_dataset/test/
  inflating: /content/Teacher_dataset/test/cache-073a0a915559c680.arrow  
  inflating: /content/Teacher_dataset/test/data-00000-of-00001.arrow  
  inflating: /content/Teacher_dataset/test/dataset_info.json  
  inflating: /content/Teacher_dataset/test/state.json  
   creating: /content/Teacher_dataset/train/
  inflating: /content/Teacher_dataset/train/cache-584a84910cca543c.arrow  
  inflating: /content/Teacher_dataset/train/data-00000-of-00001.arrow  
  inflating: /content/Teacher_dataset/train/dataset_info.json  
  inflating: /content/Teacher_dataset/train/state.json  
   creating: /content/Teacher_dataset/validation/
  inflating: /content/Teacher_dataset/validation/cache-41ad7b289e7d3c9b.arrow  
  inflating: /content/Teacher_dataset/validation/data-00000-of-00001.arrow  
  inflating: /c

In [8]:
try:
    finetuned_model = AutoModelForSeq2SeqLM.from_pretrained("distilled")
    finetuned_model = finetuned_model.to('cuda')
    print("Load success")

except:
    print("Load Failure")

Load success


In [9]:
try:
    dataset = load_from_disk("Teacher_dataset")
    print("Loaded Merged Dataset")
except:
    print("Load Failure")

Loaded Merged Dataset


In [10]:
model_name='t5-small'
tokenizer = AutoTokenizer.from_pretrained(model_name)

tokenizer_config.json:   0%|          | 0.00/2.32k [00:00<?, ?B/s]

spiece.model:   0%|          | 0.00/792k [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/1.39M [00:00<?, ?B/s]

# Import test set

In [ ]:
#Index for question selection
index = int(input("Type in the index of question to be extracted (1 to 200): "))
# Question from test set using index
question = dataset['test'][index]['question']
context = dataset['test'][index]['context']
answer = dataset['test'][index]['answer']
#Show the imported test question
print(f"Question: {question}")
print(f"Context: {context}")
print(f"Answer: {answer}")

Type in the index of question to be extracted (1 to 200): 6
Question: What was the record as of September 1?
Context: CREATE TABLE table_name_72 (record VARCHAR, date VARCHAR)
Answer: SELECT record FROM table_name_72 WHERE date = "september 1"


# Custom Question

Sample Contexts:

CREATE TABLE rec_mart_sales ("Date of Sale" date, "item name" text, "item price" number, "Customer Name" text)

CREATE TABLE rec_print_sales ("Date of Transaction" date, "No. of print pages" number, "No. of Xerox" copies number, "Student name" text, "Amount paid" number)

In [ ]:
#custom input
context = input("Write the SQL for context table creation: ")

Write the SQL for context table creation: CREATE TABLE rec_print_sales ("Date of Transaction" date, "No. of print pages" number, "No. of Xerox" copies number, "Student name" text, "Amount paid" number)


In [ ]:
question = input("What is the prompt to be asked?: ")
answer = "Flag"

What is the prompt to be asked?: When did Hari takes 10 pages print?


# Web app

##server side

In [11]:
!pip install anvil-uplink

  Using cached argparse-1.4.0-py2.py3-none-any.whl.metadata (2.8 kB)
Using cached argparse-1.4.0-py2.py3-none-any.whl (23 kB)


In [12]:
import anvil.server

In [28]:
anvil.server.connect("server_Z2VZWQTD5CQBZWLT4VBUAA42-57ADWD323GRPAAAJ")

In [52]:
@anvil.server.callable
def generate_query(question, context):

  if context:
    prompt = f"""Tables:
    {context}

    Question:
    {question}

    Answer:
    """
  else:
    def_context = """create table rec_print_sales("Date of Transaction" DATE, "Student name" VARCHAR, "Number of print pages" INT, "Number of Xerox copies" INT, "Amount paid" INT);"""
    prompt = f"""Tables:
    {def_context}

    Question:
    {question}

    Answer:
    """
  inputs = tokenizer(prompt, return_tensors='pt')
  inputs = inputs.to('cuda')

  output = tokenizer.decode(finetuned_model.generate(inputs["input_ids"],max_new_tokens=200,)[0],skip_special_tokens=True)

  conn = sqlite3.connect('test.db')
  flag=0
  try:
    df = pd.read_sql(output, conn)
  except:
    flag=1
  if flag==0:
    results=[output, flag, df]
  else:
    results=[output, flag]
  return results

In [ ]:
anvil.server.wait_forever()

#SQL side

## Setup

In [42]:
import sqlite3

In [20]:
%load_ext sql

In [21]:
%%sql
sqlite:///test.db

In [22]:
%%sql
create table rec_print_sales(Date_of_Transaction date, Student_name varchar, Number_of_print_pages int, Number_of_Xerox_copies int, Amount_paid int);

 * sqlite:///test.db
Done.


[]

In [23]:
%%sql
INSERT INTO rec_print_sales (Date_of_Transaction, Student_name, Number_of_print_pages, Number_of_Xerox_copies, Amount_paid) VALUES
('2025-03-01', 'Alice Johnson', 15, 3, 50),
('2025-03-02', 'Bob Smith', 30, 5, 100),
('2025-03-03', 'Charlie Brown', 25, 7, 90),
('2025-03-04', 'David White', 40, 2, 120),
('2025-03-05', 'Emma Clark', 10, 4, 45),
('2025-03-06', 'Fiona Harris', 20, 8, 85),
('2025-03-07', 'George Williams', 35, 6, 110),
('2025-03-08', 'Hannah Lee', 50, 10, 160),
('2025-03-09', 'Isaac Moore', 60, 5, 150),
('2025-03-10', 'Jack Taylor', 45, 12, 130),
('2025-03-11', 'Liam King', 20, 3, 60),
('2025-03-12', 'Mia Scott', 30, 5, 95),
('2025-03-13', 'Noah Lewis', 25, 2, 75),
('2025-03-14', 'Olivia Walker', 18, 4, 65),
('2025-03-15', 'Parker Harris', 28, 6, 90),
('2025-03-16', 'Quinn Evans', 36, 7, 105),
('2025-03-17', 'Ryan Mitchell', 22, 3, 70),
('2025-03-18', 'Sophia Adams', 40, 10, 125),
('2025-03-19', 'Thomas Roberts', 48, 8, 140),
('2025-03-20', 'Ursula Cooper', 50, 9, 145),
('2025-03-21', 'Victor Martin', 32, 4, 95),
('2025-03-22', 'Wendy Young', 26, 5, 80),
('2025-03-23', 'Xander Allen', 55, 10, 150),
('2025-03-24', 'Yara Gonzalez', 38, 6, 110),
('2025-03-25', 'Zachary Perez', 41, 3, 115),
('2025-03-26', 'Amelia Garcia', 29, 2, 85),
('2025-03-27', 'Benjamin Kim', 33, 4, 100),
('2025-03-28', 'Chloe Nelson', 24, 7, 85),
('2025-03-29', 'Daniel Clark', 60, 5, 155),
('2025-03-30', 'Ella Turner', 44, 8, 125),
('2025-03-31', 'Finn Walker', 53, 9, 140),
('2025-04-01', 'Grace Harris', 39, 3, 95),
('2025-04-02', 'Henry Lewis', 42, 10, 135),
('2025-04-03', 'Isla Scott', 31, 6, 100),
('2025-04-04', 'James Moore', 27, 5, 85),
('2025-04-05', 'Katherine Brown', 22, 4, 75),
('2025-04-06', 'Leo White', 19, 7, 60),
('2025-04-07', 'Maya Johnson', 36, 6, 110),
('2025-04-08', 'Nina King', 48, 5, 140),
('2025-04-09', 'Oscar Adams', 41, 3, 125),
('2025-04-10', 'Paige Taylor', 28, 9, 95),
('2025-04-11', 'Quincy Robinson', 52, 8, 145),
('2025-04-12', 'Riley Young', 39, 6, 115),
('2025-04-13', 'Sophia Martin', 33, 4, 95),
('2025-04-14', 'Travis Hall', 47, 10, 150),
('2025-04-15', 'Uma Lee', 21, 2, 60),
('2025-04-16', 'Violet Mitchell', 29, 7, 85),
('2025-04-17', 'Willow Roberts', 34, 3, 95),
('2025-04-18', 'Xander Moore', 45, 9, 130),
('2025-04-19', 'Yasmine Green', 38, 4, 105),
('2025-04-20', 'Zane White', 50, 5, 140);


 * sqlite:///test.db
51 rows affected.


[]

In [43]:
conn = sqlite3.connect('test.db')

## Running SQL

In [44]:
query = 'SELECT "Date_of_Transaction" FROM rec_print_sales WHERE "Number_of_Xerox_copies" = 3'

In [47]:
df = pd.read_sql(query, conn)

In [46]:
df

,Date_of_Transaction
0,2025-03-01
1,2025-03-11
2,2025-03-17
3,2025-03-25
4,2025-04-01
5,2025-04-09
6,2025-04-17
